In [1]:
import torch
import matplotlib.pyplot as plt
from dataset_sampler import create_quick_test_dataset, cleanup_test_dataset, show_dataset_stats
from quick_test import run_quick_test, run_ultra_quick_test, run_medium_test

print("🚀 NOTEBOOK DE TESTE RÁPIDO CARREGADO!")
print("=" * 40)

# ===================================================================
# CÉLULA 2: Teste Ultra Rápido (2 minutos)
# ===================================================================

print("⚡ EXECUTANDO TESTE ULTRA RÁPIDO...")
print("📊 50 samples, 2 epochs, batch_size=2")
print("⏱️ Tempo estimado: ~2 minutos")

# Executar teste ultra rápido
model, dataset_dir = run_ultra_quick_test()

print("✅ Teste ultra rápido concluído!")

# ===================================================================
# CÉLULA 3: Criar dataset de teste personalizado
# ===================================================================

# Opção 1: Dataset pequeno para debug
print("📁 Criando dataset de teste personalizado...")

# Criar dataset com 100 imagens
test_dataset = create_quick_test_dataset(samples=100)
show_dataset_stats(test_dataset)

# ===================================================================
# CÉLULA 4: Teste com configurações personalizadas
# ===================================================================

from config import *
from dataset import create_dataloaders
from model import create_model
from trainer import train_model
from utils import get_device, print_model_info

# Configurações de teste
TEST_CONFIG = {
    'samples': 150,
    'epochs': 8,
    'batch_size': 6,
    'learning_rate': 0.0005
}

print(f"🔧 Configurações de teste: {TEST_CONFIG}")

# Criar dataset de teste
test_dir = create_quick_test_dataset(samples=TEST_CONFIG['samples'])

# Setup
device = get_device()
train_loader, val_loader = create_dataloaders(
    dataset_dir=test_dir,
    batch_size=TEST_CONFIG['batch_size'],
    img_size=IMG_SIZE,
    num_workers=2
)

model = create_model(num_classes=NUM_CLASSES, device=device)
print_model_info(model)

# Treinamento
trained_model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=TEST_CONFIG['epochs'],
    learning_rate=TEST_CONFIG['learning_rate'],
    device=device,
    save_path='custom_test_model.pth'
)

print("✅ Teste personalizado concluído!")

# ===================================================================
# CÉLULA 5: Análise dos resultados
# ===================================================================

# Testar inferência com algumas imagens
trained_model.eval()

# Pegar um batch de teste
sample_batch = next(iter(val_loader))
images, targets = sample_batch

with torch.no_grad():
    predictions = trained_model(images.to(device))
    
print(f"📊 Análise dos resultados:")
print(f"   Input shape: {images.shape}")
print(f"   Predictions shape: {predictions.shape}")
print(f"   Número de targets: {len(targets)}")

# Visualizar uma imagem (opcional)
"""
import matplotlib.pyplot as plt
import numpy as np

# Desnormalizar imagem para visualização
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

# Mostrar primeira imagem
img = denormalize(images[0]).permute(1, 2, 0).numpy()
img = np.clip(img, 0, 1)

plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title("Imagem de Teste")
plt.axis('off')
plt.show()
"""

# ===================================================================
# CÉLULA 6: Comparação de velocidade
# ===================================================================

import time

def benchmark_inference(model, dataloader, device, num_batches=10):
    """Mede velocidade de inferência"""
    model.eval()
    
    times = []
    with torch.no_grad():
        for i, (images, _) in enumerate(dataloader):
            if i >= num_batches:
                break
                
            start_time = time.time()
            _ = model(images.to(device))
            end_time = time.time()
            
            times.append(end_time - start_time)
    
    avg_time = sum(times) / len(times)
    fps = images.size(0) / avg_time  # frames per second
    
    return avg_time, fps

# Benchmark do modelo
avg_time, fps = benchmark_inference(trained_model, val_loader, device)

print(f"⚡ BENCHMARK DE PERFORMANCE:")
print(f"   Tempo médio por batch: {avg_time:.4f}s")
print(f"   FPS (frames per second): {fps:.2f}")
print(f"   Batch size: {TEST_CONFIG['batch_size']}")

# ===================================================================
# CÉLULA 7: Limpeza (opcional)
# ===================================================================

# Descomente para limpar os datasets de teste
"""
cleanup_test_dataset(test_dataset)
cleanup_test_dataset(test_dir)
print("🧹 Datasets de teste removidos!")
"""

print("\n" + "="*50)
print("🎉 TODOS OS TESTES CONCLUÍDOS!")
print("="*50)
print("✅ Pipeline funcionando corretamente")
print("✅ Pronto para treinamento completo")
print("💡 Para treinamento completo, use: %run main.py")

# ===================================================================
# CÉLULA 8: Funções utilitárias para testes rápidos
# ===================================================================

def quick_debug_test():
    """Teste super rápido para debug"""
    print("🐛 TESTE DE DEBUG (20 samples, 1 epoch)")
    return run_quick_test(samples=20, epochs=1, batch_size=2)

def validate_pipeline():
    """Valida se o pipeline está funcionando"""
    try:
        quick_debug_test()
        print("✅ Pipeline validado com sucesso!")
        return True
    except Exception as e:
        print(f"❌ Erro na validação: {e}")
        return False

def memory_test():
    """Testa uso de memória"""
    if torch.cuda.is_available():
        print(f"🔋 Memória GPU antes: {torch.cuda.memory_allocated()//1024//1024}MB")
        
        # Criar modelo temporário
        temp_model = create_model(device='cuda')
        temp_images = torch.randn(4, 3, IMG_SIZE, IMG_SIZE).cuda()
        
        with torch.no_grad():
            _ = temp_model(temp_images)
        
        print(f"🔋 Memória GPU depois: {torch.cuda.memory_allocated()//1024//1024}MB")
        
        # Limpeza
        del temp_model, temp_images
        torch.cuda.empty_cache()
        
        print(f"🔋 Memória GPU após limpeza: {torch.cuda.memory_allocated()//1024//1024}MB")
    else:
        print("💻 Usando CPU - teste de memória não aplicável")

# Executar validação
print("\n🔍 VALIDANDO PIPELINE...")
pipeline_ok = validate_pipeline()

if pipeline_ok:
    print("🎯 Pronto para começar o desenvolvimento!")
else:
    print("⚠️ Verifique os erros antes de continuar")

🚀 NOTEBOOK DE TESTE RÁPIDO CARREGADO!
⚡ EXECUTANDO TESTE ULTRA RÁPIDO...
📊 50 samples, 2 epochs, batch_size=2
⏱️ Tempo estimado: ~2 minutos
⚡ TESTE ULTRA RÁPIDO (50 samples, 2 epochs)
🚀 INICIANDO TESTE RÁPIDO DO PIPELINE

1️⃣ Criando dataset de teste...
🚀 CRIANDO DATASET DE TESTE RÁPIDO
📁 Criando dataset de teste em: dataset_test
📊 Amostras de treino: 40
📊 Amostras de validação: 10
--------------------------------------------------
✅ Train: 40 amostras copiadas
✅ Val: 10 amostras copiadas
--------------------------------------------------
✅ Dataset de teste criado com sucesso!
📍 Localização: dataset_test

📊 ESTATÍSTICAS DO DATASET:
------------------------------
Train:  40 imagens,  40 labels
  Val:  10 imagens,  10 labels
------------------------------
Total:  50 imagens,  50 labels

💡 COMO USAR:
1. Importe: from config import *
2. Mude: DATASET_DIR = 'dataset_test'
3. Execute seu treinamento normalmente!

⚡ Treinamento será muito mais rápido para testes!

2️⃣ Configurando dispositivo

KeyboardInterrupt: 